# Диагностика и анализ пайплайна

В этом ноутбуке представлены инструменты для диагностики, визуализации и анализа результатов дубляжа.

In [1]:
# --- Импорт библиотек ---
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio, Video
import json

## 1. Визуализация сегментов (таймлиния)
Загрузим сегменты с таймкодами и построим таймлинию.

In [ ]:
import os
import glob
import re

output_dir = '../data/output'
pattern = os.path.join(output_dir, 'asr_segments_ru_*.txt')
files = glob.glob(pattern)

print("Найдено файлов:")
for f in files:
    print(f"{os.path.basename(f)} — {os.path.getsize(f)} байт")

print("\nПроверка регулярки:")
for f in files:
    fname = os.path.basename(f)
    match = re.match(r'asr_segments_ru_\d{8}_\d{4}\.txt$', fname)
    print(f"{fname}: match={bool(match)}, size={os.path.getsize(f)}")

print("\nПроверка startswith/endswith:")
for f in files:
    fname = os.path.basename(f)
    if fname.startswith('asr_segments_ru_') and fname.endswith('.txt'):
        print(f"{fname} — подходит по startswith/endswith")

print("\nПроверка без ограничения по размеру:")
valid_files = []
for f in files:
    fname = os.path.basename(f)
    if re.match(r'asr_segments_ru_\d{8}_\d{4}\.txt$', fname):
        valid_files.append(f)
print("Файлы, прошедшие регулярку:", valid_files)

In [ ]:
import os
import glob
import re

output_dir = '../data/output'
pattern = os.path.join(output_dir, 'asr_segments_ru_*.txt')
files = glob.glob(pattern)

# Фильтруем только те, что >1 КБ и имя строго по шаблону с датой/временем
valid_files = []
for f in files:
    fname = os.path.basename(f)
    if re.match(r'asr_segments_ru_\d{8}_\d{4}\.txt$', fname) and os.path.getsize(f) > 1024:
        valid_files.append(f)

if not valid_files:
    raise FileNotFoundError(f'Не найден ни один валидный файл перевода сегментов в {output_dir}/')

def extract_dt(fname):
    m = re.search(r'(\d{8}_\d{4})', fname)
    return m.group(1) if m else ''

valid_files.sort(key=lambda f: extract_dt(os.path.basename(f)), reverse=True)
path = valid_files[0]

print(f'Используется файл перевода: {path}')
with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
segments = []

## 2. Прослушивание отдельных сегментов
Выберите номер сегмента для прослушивания.

In [ ]:
# Для сегмента TTS
seg_idx = 0  # измените на нужный номер
Audio('../data/temp/run_tts_segment_{}.wav'.format(seg_idx))

## 3. Сравнение оригинального и итогового аудио
Воспроизведём оба файла для сравнения.

In [ ]:
# Оригинальное аудио (извлечённое из видео)
print('Оригинальное аудио:')
Audio('../data/temp/run_extracted_audio.wav')

In [ ]:
# Итоговое аудио (склеенное)
print('Итоговое аудио:')
Audio('../data/temp/run_assembled_audio.wav')

## 4. Визуализация ошибок и длительности сегментов
Построим график длительности и отметим сегменты, где длительность сильно отличается от оригинала.

In [ ]:
# Для примера: сравним длительность оригинальных и сгенерированных сегментов
orig_durations = [seg['end'] - seg['start'] for seg in segments]
gen_durations = []
import os
from pydub import AudioSegment
for i in range(len(segments)):
    path = f'../data/temp/run_tts_segment_{i}.wav'
    if os.path.exists(path):
        audio = AudioSegment.from_file(path)
        gen_durations.append(len(audio)/1000)
    else:
        gen_durations.append(0)
plt.figure(figsize=(10,4))
plt.plot(orig_durations, label='Оригинал')
plt.plot(gen_durations, label='Сгенерировано')
plt.xlabel('Сегмент')
plt.ylabel('Длительность, сек')
plt.legend()
plt.title('Сравнение длительности сегментов')
plt.show()

## 5. Видеоплеер для итогового видео

In [ ]:
import glob
from IPython.display import Video
import os
import re

output_dir = '../data/output'
# Ищем все mp4-файлы, начинающиеся с output
files = glob.glob(os.path.join(output_dir, 'output*.mp4'))
if not files:
    raise FileNotFoundError('Не найдено ни одного итогового видео в ../data/output/')
# Сортируем по времени изменения (или по дате/времени в имени, если нужно)
files.sort(key=os.path.getmtime, reverse=True)
video_path = files[0]
print(f'Показывается видео: {video_path}')
Video(video_path, embed=True)